In [153]:
import wget

In [154]:
# url = "https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py"
# wget.download(url)

In [155]:
#Docoment.json download
# url="https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json"
# wget.download(url)

In [156]:
import minsearch

In [157]:
import json

In [158]:
with open('documents.json','rt') as f_in:
    docs_raw=json.load(f_in)

In [159]:
documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course']=course_dict['course']
        documents.append(doc)


Index the Document

In [160]:
index =minsearch.Index(
    text_fields=["question","text","section"],
    keyword_fields=["course"]

)

In [161]:
index.fit(documents)


In [162]:
#q='the course has already started, can i still enroll?'

In [163]:
from openai import OpenAI

In [164]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")




In [165]:
client = OpenAI(api_key=api_key)

In [166]:
def search(query):
    boost={'question':3.0,'section':0.5}

    results=index.search(
        query=query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
)
    return results

In [173]:
def build_prompt(query,search_results):
    prompt_template="""
You're a course teaching assistance. Answer the QUESTION based on the CONTEXT from the FAQ Databalse. Use only the facts from the CONTEXT when answering the QUESTION.


Question :{question}

CONTEXT:
{context}
""".strip()


    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"


    prompt=prompt_template.format(question=query,context=context).strip()
    return prompt

In [175]:
def llm(prompt):
    response=client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":prompt}]
)
    return response.choices[0].message.content

In [ ]:
def rag(query):
    search_results=search(query)
    prompt=build_prompt(query,search_results)
    answer=llm(prompt)
    return answer

In [183]:
rag("the course has already started. Can i still enroll?")

"Yes, you can still enroll after the course has started. You are eligible to submit the homework assignments. However, be mindful that there will be deadlines for submitting the final projects, so it's important not to leave everything until the last minute."